# Introduce


## PPO

![grpo](./images/open_ai_ppo.png)


## GRPO

![grpo](./images/grpo.png)

## GRPO实践-复现deepseek-r1

### 架构

```
┌─────────────────────────────────────────────────────────────-──┐
│                        GRPO 架构                                │
└────────────────────────────────────────────────────────────────┘
                            │
            ┌───────────────┴───────────────┐
            ▼                               ▼
┌───────────────────────┐       ┌────────────────────────┐
│    生成服务 (vLLM)     │       │  参考模型服务 (ref_server)│
│                       │       │                        │
│  ┌─────────────────┐  │       │  ┌──────────────────┐  │
│  │ 问题抽样与生成    │  │       │  │ 参考模型 (固定)   │  │
│  └─────────────────┘  │       │  └──────────────────┘  │
│          │            │       │          │             │
│          ▼            │       │          ▼             │
│  ┌─────────────────┐  │       │  ┌──────────────────┐  │
│  │ 奖励计算         │  │       │  │ 参考logprobs计算  │  │
│  └─────────────────┘  │       │  └──────────────────┘  │
│          │            │       │          │             │
└───────────┼────────────┘       └──────────┼─────────────┘
            │                               │
            └───────────────┬───────────────┘
                            ▼
               ┌──────────────────────────┐
               │     训练服务 (DeepSpeed)  │
               │                          │
               │  ┌────────────────────┐  │
               │  │  训练模型 (policy)  │  │
               │  └────────────────────┘  │
               │           │              │
               │           ▼              │
               │  ┌────────────────────┐  │
               │  │   GRPO_step 更新   │  │
               │  └────────────────────┘  │
               │           │              │
               │           ▼              │
               │  ┌────────────────────┐  │
               │  │    模型状态同步     │  │
               │  └────────────────────┘  │
               └──────────────────────────┘
```


- 生成服务: 计算策略梯度
- 参考模型: 计算KL散度

### 流程

```mermaid
sequenceDiagram
    participant T as 训练服务 (DeepSpeed)
    participant G as 生成服务 (vLLM)
    participant R as 参考模型服务 (ref_server)
    
    Note over T,R: 系统初始化
    T->>G: 创建生成进程 (mp.Process)
    G->>G: 初始化vLLM模型
    T->>T: 初始化DeepSpeed和训练模型
    
    loop 训练循环 (1 到 all_steps)
        T->>R: 请求批次数据 (get_batch)
        
        Note over G: 数据生成阶段
        G->>G: 从GSM8K随机抽取问题
        G->>G: 使用vLLM生成多个回答 (num_pre_Q)
        G->>G: 计算奖励 (reward_correct + reward_format)
        
        G->>R: 上传生成的样本和奖励 (/upload)
        
        Note over R: 参考模型计算阶段
        R->>R: 使用参考模型计算token级log概率
        R->>R: 将数据放入结果队列
        
        R-->>T: 返回批次数据 (/get)
        
        Note over T: 训练阶段
        T->>T: 执行GRPO_step计算损失
        T->>T: 执行backward和optimizer.step
        
        alt 每gen_update_steps步
            T->>G: 更新生成服务模型参数 (Q.put)
            G->>G: 更新vLLM模型
        end
        
        alt 每save_steps步
            T->>T: 保存模型检查点
        end
    end
```

### 运行

1. CUDA_VISIBLE_DEVICES=7 ref_server.py
2. CUDA_VISIBLE_DEVICES=2,3,4,5,6 deepspeed grpo.py
